In [3]:
import numpy as np
import pandas as pd
from obspy import read
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from obspy.signal.trigger import classic_sta_lta, trigger_onset
from scipy import signal

class SeismicDataProcessor:
    def __init__(self, data_directory=r'C:\Users\salos\OneDrive\Desktop\hacking-nasa-with-html\space_apps_2024_seismic_detection\data\lunar\training'):
        self.data_directory = data_directory
        self.catalog_directory = f'{data_directory}catalogs/'
        self.data_subdirectory = f'{data_directory}data/S12_GradeA/'
        
    def load_catalog(self, catalog_name='apollo12_catalog_GradeA_final.csv'):
        """Load the seismic event catalog"""
        catalog_path = self.catalog_directory + catalog_name
        return pd.read_csv(catalog_path)
    
    def load_seismic_data(self, filename):
        """Load seismic data from miniseed file"""
        mseed_file = f'{self.data_subdirectory}{filename}.mseed'
        return read(mseed_file)
    
    def extract_features(self, stream, arrival_time_rel, window_before=60, window_after=60):
        """Extract features from seismic data around arrival time"""
        tr = stream[0]
        data = tr.data
        times = tr.times()
        
        # Find indices for the window around arrival
        sample_rate = tr.stats.sampling_rate
        arrival_index = int(arrival_time_rel * sample_rate)
        window_before_samples = int(window_before * sample_rate)
        window_after_samples = int(window_after * sample_rate)
        
        start_idx = max(0, arrival_index - window_before_samples)
        end_idx = min(len(data), arrival_index + window_after_samples)
        
        # Extract windowed data
        window_data = data[start_idx:end_idx]
        window_times = times[start_idx:end_idx]
        
        # Calculate STA/LTA
        sta_len = 120  # Short-term window length in seconds
        lta_len = 600  # Long-term window length in seconds
        cft = classic_sta_lta(window_data, 
                             int(sta_len * sample_rate), 
                             int(lta_len * sample_rate))
        
        # Calculate spectrogram
        f, t, sxx = signal.spectrogram(window_data, sample_rate)
        
        return {
            'waveform': window_data,
            'times': window_times,
            'sta_lta': cft,
            'spectrogram': {
                'frequencies': f,
                'times': t,
                'power': sxx
            }
        }
    
    def prepare_training_data(self):
        """Prepare training dataset from all available events"""
        catalog = self.load_catalog()
        training_data = []
        
        for _, row in catalog.iterrows():
            filename = row['filename']
            arrival_time_rel = row['time_rel(sec)']
            
            try:
                stream = self.load_seismic_data(filename)
                features = self.extract_features(stream, arrival_time_rel)
                
                training_data.append({
                    'filename': filename,
                    'arrival_time': arrival_time_rel,
                    'features': features
                })
            except Exception as e:
                print(f"Error processing {filename}: {e}")
        
        return training_data

# Example usage
def main():
    processor = SeismicDataProcessor()
    training_data = processor.prepare_training_data()
    
    print(f"Processed {len(training_data)} seismic events")
    
    # Example: Plot the first event
    if training_data:
        event = training_data[0]
        
        plt.figure(figsize=(12, 8))
        
        # Plot waveform
        plt.subplot(211)
        plt.plot(event['features']['times'], event['features']['waveform'])
        plt.axvline(x=event['arrival_time'], color='red', label='Arrival Time')
        plt.legend()
        plt.title(f"Seismic Waveform - {event['filename']}")
        
        # Plot STA/LTA
        plt.subplot(212)
        plt.plot(event['features']['times'], event['features']['sta_lta'])
        plt.axvline(x=event['arrival_time'], color='red', label='Arrival Time')
        plt.legend()
        plt.title("STA/LTA Characteristic Function")
        
        plt.tight_layout()
        plt.show()

if __name__ == "__main__":
    main()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\salos\\OneDrive\\Desktop\\hacking-nasa-with-html\\space_apps_2024_seismic_detection\\data\\lunar\\trainingcatalogs/apollo12_catalog_GradeA_final.csv'